In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np


In [2]:
VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)


In [3]:
len(train_data[1])

189

In [4]:
train_data=sequence.pad_sequences(train_data,MAXLEN)
test_data=sequence.pad_sequences(test_data,MAXLEN)
len(train_data[1])


250

In [5]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE,32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['accuracy'])
history=model.fit(train_data,train_labels,epochs=10,validation_split=0.2)


Epoch 1/10
625/625 [==============================] - 157s 237ms/step - loss: 0.4455 - accuracy: 0.7885 - val_loss: 0.2929 - val_accuracy: 0.8818
Epoch 2/10
625/625 [==============================] - 191s 306ms/step - loss: 0.2571 - accuracy: 0.9017 - val_loss: 0.2877 - val_accuracy: 0.8904
Epoch 3/10
625/625 [==============================] - 186s 298ms/step - loss: 0.2010 - accuracy: 0.9275 - val_loss: 0.3046 - val_accuracy: 0.8718
Epoch 4/10
625/625 [==============================] - 191s 305ms/step - loss: 0.1588 - accuracy: 0.9441 - val_loss: 0.3473 - val_accuracy: 0.8842
Epoch 5/10
625/625 [==============================] - 184s 295ms/step - loss: 0.1347 - accuracy: 0.9552 - val_loss: 0.3393 - val_accuracy: 0.8880
Epoch 6/10
625/625 [==============================] - 167s 267ms/step - loss: 0.1118 - accuracy: 0.9632 - val_loss: 0.3241 - val_accuracy: 0.8760
Epoch 7/10
625/625 [==============================] - 163s 261ms/step - loss: 0.0945 - accuracy: 0.9704 - val_loss: 0.3441 -

In [7]:
model.save("lstm.h5")
new_model = tf.keras.models.load_model('lstm.h5')
results=new_model.evaluate(test_data,test_labels)
print(results)


C:\Users\windows\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 58s 72ms/step - loss: 0.4520 - accuracy: 0.8595
[0.45200157165527344, 0.8594800233840942]


In [8]:
results=model.evaluate(test_data,test_labels)
print(results)

782/782 [==============================] - 60s 77ms/step - loss: 0.4520 - accuracy: 0.8595
[0.45200157165527344, 0.8594800233840942]


In [9]:
word_index=imdb.get_word_index()


In [10]:
for i in range(10):
    print(list(word_index.keys())[i],':',list(word_index.values())[i])
fawn : 34701
tsukino : 52006
nunnery : 52007
sonja : 16816
vani : 63951
woods : 1408
spiders : 16115
hanging : 2345
woody : 2289
trawling : 52008
def encode_text(text):
    tokens=keras.preprocessing.text.text_to_word_sequence(text)
    tokens=[word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens],MAXLEN)[0]
text="that movie was amazing, i have to watch it again"
encoded=encode_text(text)
print(encoded)


fawn : 34701
tsukino : 52006
nunnery : 52007
sonja : 16816
vani : 63951
woods : 1408
spiders : 16115
hanging : 2345
woody : 2289
trawling : 52008
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [11]:
reverse_word_index={value:key for (key,value) in word_index.items()}

def decode_integers(integers):
    PAD=0
    text=""
    for num in integers:
        if num!=PAD:
            text+=reverse_word_index[num] +" "
            
    return text[:-1]

print(decode_integers(encoded))


that movie was amazing i have to watch it again


In [12]:
def predict(text):
    encoded_text=encode_text(text)
    pred=encoded_text.reshape(1,250) #converting vector to 2d
    result=model.predict(pred)
    print(result[0])
positive_review="That was a good movie, i will definitely watch it again"
predict(positive_review)

negative_review="Don't waste your time watching this movie, so disappointing"
predict(negative_review)

1/1 [==============================] - 3s 3s/step
[0.96824074]
1/1 [==============================] - 0s 157ms/step
[0.3615175]


In [13]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 2s 2us/step


In [15]:
text=open(path_to_file,'rb').read().decode(encoding='utf-8')
print("Length of text : ",len(text))
Length_of_text :  1115394
print(text[:250])


Length of text :  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [16]:
vocab=sorted(set(text))

In [22]:
char2idx={u:i for i,u in enumerate(vocab)}
idx2char=np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[t] for t in text])

text_as_int=text_to_int(text)
print('Text:',text[0:13])
print('Encoded:',text_to_int(text[:13]))


Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [23]:
def int_to_text(ints):
    try:
        ints=ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

print(int_to_text(text_to_int(text[:13])))


First Citizen


In [24]:
seq_length = 100 
examples_per_epoch = len(text)//(seq_length+1)


In [25]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):  
    input_text = chunk[:-1]  
    target_text = chunk[1:]  
    return input_text, target_text  

dataset = sequences.map(split_input_target) 
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))




EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [37]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=[batch_size, None]),
      tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
    ])
    return model

model=build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)  
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  
print(example_batch_predictions.shape)
print(len(example_batch_predictions))
print(example_batch_predictions)


(64, 100, 65) # (batch_size, sequence_length, vocab_size)
(64, 100, 65)
64
tf.Tensor(
[[[ 1.3251001e-03  3.4430767e-03  2.0326218e-03 ... -5.0887601e-03
   -5.8780783e-03  2.6177586e-04]
  [ 2.1975916e-03  6.5987865e-03  9.6542435e-04 ... -3.8792577e-03
   -1.6570603e-03  7.6554157e-04]
  [ 6.0186153e-03 -4.9537304e-04  8.0355564e-03 ... -6.1021307e-03
   -2.4897940e-03 -4.0712054e-03]
  ...
  [ 2.2069428e-02  9.4147148e-03  3.7962091e-03 ...  1.7359682e-03
   -2.0415823e-03 -5.5994829e-03]
  [ 1.2643751e-02  5.2989228e-03  8.3993217e-03 ...  7.6447381e-04
    4.3322295e-03 -4.2372751e-03]
  [ 9.3948161e-03  2.2232563e-03  1.9938436e-03 ...  2.6804060e-03
   -1.5712748e-03 -4.9629519e-03]]

 [[-2.4788573e-03 -1.9733938e-03 -4.8176409e-03 ...  1.9501196e-03
   -4.3313950e-03 -5.3163874e-04]
  [ 1.1893604e-03 -2.1203582e-03 -3.2369792e-04 ... -2.5113593e-03
   -6.2564849e-03  2.3356115e-03]
  [ 1.1956047e-03  4.9231229e-03 -1.6232933e-03 ... -5.6215636e-03
   -7.7098832e-03  4.9132360e-0

In [39]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)


100
tf.Tensor(
[[ 0.0013251   0.00344308  0.00203262 ... -0.00508876 -0.00587808
   0.00026178]
 [ 0.00219759  0.00659879  0.00096542 ... -0.00387926 -0.00165706
   0.00076554]
 [ 0.00601862 -0.00049537  0.00803556 ... -0.00610213 -0.00248979
  -0.00407121]
 ...
 [ 0.02206943  0.00941471  0.00379621 ...  0.00173597 -0.00204158
  -0.00559948]
 [ 0.01264375  0.00529892  0.00839932 ...  0.00076447  0.00433223
  -0.00423728]
 [ 0.00939482  0.00222326  0.00199384 ...  0.00268041 -0.00157127
  -0.00496295]], shape=(100, 65), dtype=float32)


In [40]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)


65
tf.Tensor(
[ 1.32510008e-03  3.44307674e-03  2.03262176e-03  1.85602531e-03
  2.21742853e-03  2.53064930e-03 -1.65632018e-03  3.88318626e-03
 -5.86581766e-04  2.57285684e-03  2.26133107e-03 -2.53481371e-03
  6.08188566e-03  2.48184148e-03  1.77756604e-03 -6.78903144e-03
  7.00137112e-04 -3.31419054e-03 -2.14862870e-03 -3.74247378e-04
 -9.56521253e-04  1.17776846e-03 -8.17543268e-03 -5.03043924e-03
 -5.11549762e-04  1.95413129e-03  5.36165759e-03 -1.21163984e-03
 -1.52174174e-03  2.20200920e-04 -5.00089489e-03  1.55254756e-03
 -1.82190724e-03  1.23672618e-03 -1.59741449e-03 -3.95397050e-03
  1.46192324e-03 -2.33040797e-03 -4.10161028e-03 -2.93789781e-04
 -4.31525335e-03 -6.02600456e-04  6.84375875e-04  3.74924159e-04
  8.22579302e-03  3.40878684e-03 -1.05858105e-03 -3.29450145e-03
 -5.20249596e-04  1.26444465e-02  2.22149701e-03  5.40079037e-03
 -1.11500034e-04  2.68878159e-03 -4.21652244e-03  1.44922850e-03
  2.36101379e-03 -3.58467922e-03  3.14204022e-03  3.82014504e-03
  5.9227983

In [41]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

In [42]:
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars 


"MmtHDj! lW3PfB&Y heRmoSt dWStS3'ZPRgXC.N t?sbBs'cGDmVtouZqp bIEq$poJfOtk!B$AatsKeJ:re$?z'ZqYroM,aJe$"

In [43]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)


In [44]:
checkpoint_dir = './training_checkpoints'

In [46]:
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
history = model.fit(data, epochs=2, callbacks=[checkpoint_callback])

Epoch 1/2
172/172 [==============================] - 4990s 29s/step - loss: 2.5741
Epoch 2/2
172/172 [==============================] - 4970s 29s/step - loss: 1.8780


In [47]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))


In [48]:
def generate_text(model, start_string):
    num_generate = 800
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1.0
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))
inp="Romeo said"
print(generate_text(model,inp))


Romeo said.

PLBRYET
Hus by the nable and rended gouser farse.

MUKINGBE:
God I am stort of her.

Fwast of RPiRES:
Io mast you would ecress'd shose it Carinam!

All:
In cexs; fouch alle fires fither.

LAVY:
Bustak' dight nome. he colllen, gearow liniom
Mistarralist when that dive foung offreach his,
BefI Then Jusines is Gothing trared for the hone.

Plotter:
These news your iences, In.

TRANIO:
Clove, seg,
Live me hus gors; I so not with of a leady
Nore not soen sovelf
Is balla will me to far to that wean.

Ammant:
Cood with sw'lv all well me and neart?

First Kerether:
Hith so you ach well, our donghol surs, mugts infucest your that I will
Sto befose than amper we dearn,
Toums, for the marise, noverd we and likes,
Camy!

Binot Sitizen:
The keot all besty musting bihe, id yourd rove and than A alur-
